**Show all your work for full credit. Each source code you submit should include detailed comments and instructions on how to run it in order to confirm that it works as expected. If the program that does not run or throws runtime errors, it cannot be graded. You can refer to the programming guidelines from the TAs here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

**This is an individual assignment and each student should work on their own. Ensure you don't share any code online or with others (note, using Replit, GitHub and similar online platforms can make your code accessible to others).**

**To submit the assignment, you need to use Jupyter Notebook with the provided cell blocks and follow the naming conventions and instructions posted here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and section below:

<font color='red' size="4">Import any additional libraries you need in the same code block that you use it.</font>

In [ ]:
NAME = "Shruthilaya Arun"
#SECTION = "472"
SECTION = "672"

---

## Question 2 [50 points total - answer all parts] In this question you will implement a variant of the GCM-SIV cipher mode (see https://eprint.iacr.org/2015/102.pdf, especially page 8) using Simon64/128 as the block cipher, as well as a C-W MAC based on a PRP (use Simon64/128) and a poly-based UH (a 64-bit variant of GHASH that we will call BHASH64 for clarity). </font>

In [ ]:
from Crypto.Random import get_random_bytes
from simon import SimonCipher
from Crypto.Util.number import bytes_to_long , long_to_bytes

from numpy.polynomial import polynomial as Poly
import numpy as np

In [61]:
from Cryptodome.Random import get_random_bytes
from simon import SimonCipher
from Cryptodome.Util.number import bytes_to_long , long_to_bytes

from numpy.polynomial import polynomial as Poly
import numpy as np

## (a) [15 points] Implement Simon64/128-CTR(K1,N1,M) using a python crypto library in a secure way. Your implementation should support multiple plaintext blocks and also receive a nonce N1 and key K1 as inputs. Include decryption support.

## What are the bitsizes of N1 and K1? Implement the correct sizes in code

In [83]:
# 2 points
def gen_nonce() -> bytes:
    """
    generate nonce of correct size.
    """
    nonce = b""
    # YOUR CODE HERE
    nonce=get_random_bytes(8) # geneate nonce
    return nonce

# 2 points
def gen_key() -> bytes:
    """
    generate key of correct size.
    """
    key = b""
    # YOUR CODE HERE 
    key=get_random_bytes(16) # generate key
    return key

In [84]:
nonce = gen_nonce()
assert type(nonce) == bytes

# hidden tests

In [85]:
key = gen_key()
assert type(key) == bytes

# hidden tests

In [86]:
# 6 points
def simon_encrypt(key: int, nonce: int, ptxt: bytes) -> bytes:
    """
    Implement Simon64/128-CTR encryption. Initialize counter to 1.
    Return ctxt.
    Use long_to_bytes() and bytes_to_long() for bytes <-> int conversions
    """
    ctxt = b""
    #nonce = b""
    # YOUR CODE HERE
    cipher=SimonCipher(key, key_size=128, block_size=64, mode='CTR',init=nonce,counter=1) # simon 64/128 ctr mode
    block_size=8 # block
    for i in range(0, len(ptxt), block_size):
        block=ptxt[i:i + block_size] # get block
        ctxt+=long_to_bytes(cipher.encrypt(bytes_to_long(block)))
    return ctxt
# 5 points
def simon_decrypt(key: int, nonce: int, ctxt: bytes) -> bytes:
    """
    Implement Simon64/128-CTR decryption. Initialize counter to 1.
    Return ptxt.
    Use long_to_bytes() and bytes_to_long() for bytes <-> int conversions
    """
    ptxt = b""
    # YOUR CODE HERE
    cipher=SimonCipher(key,key_size=128,block_size=64,mode='CTR',init=nonce,counter=1) 
    block_size=8# block size
    for i in range(0,len(ctxt),block_size):
        block=ctxt[i:i+block_size] # get block
        decrypted_block=long_to_bytes(cipher.decrypt(bytes_to_long(block)),block_size) # decrypt block and add to ptxt
        #print(f"Block {i // block_size + 1}: Counter = {cipher.counter}, Decrypted block = {decrypted_block}")
        ptxt+=decrypted_block
    return ptxt


In [87]:
ptxt = b"I'm a good student that createsptxtsize of more than 16"
key = 333349676194713905035426349908913194778
nonce = 17345587175309373759

ctxt = simon_encrypt(key, nonce, ptxt)
assert ctxt == b'*t\xf2\xb7\r\xb1\x00fp\x8a \xa5 \x0eh#q\xf1\x1e?[\xc1\x03\x08\x87nG\xea\xb2E\xf7\x80Y\t`c\x80\xfa6\x1b\x1e\x18\xf8\xd2\xe8\xcd\xebR\xb7y\xeb\xa8\xef3\x07/'

In [88]:
ctxt = b'\xfb\xa3\rJD1\x94r\xf6;_\x92\xddM\xab\xfa%h\xfcu'
key = 333349676194713905035426349908913194778
nonce = 17345587175309373759
ptxt = simon_decrypt(key, nonce, ctxt)
assert ptxt == b'\x98\xf0\x92\xdd(\xa0\xf3{\xe9\xd5_D\x896\xa7\xbc\x1f\x85>K\x16\xc8\x8b]'

## (b) [15 points] Review the relevant code samples on Canvas and implement the universal hash BHASH64(s,A,M) that uses the irreducible polynomial P(X)=X^64+1 and s=Simon64/128(K2,0x00) (Single block ECB). Input A is the associated data blocks and input M is the plaintext blocks. After processing A and M, BHASH64 processes one additional 64-bit block so that the 32MSBs encode the number of blocks of A and the 32 LSBs encode the number of blocks of M.

In [89]:
# Given helper functions from Canvas
def modCoeffs(poly, n):
    '''
    Apply a "mod n" operation to each coefficient in a list
    The polynomial is expressed as a list of coefficients
    '''
    for i in range(len(poly)):
        poly[i] %= n
    return poly

def poly_xor (poly_1,poly_2):
    return modCoeffs(poly_1+poly_2,2)

def poly_mul(poly_1, poly_2, gf_degree, modulus):

    # if len(poly_1)!=gf_degree:
    #     poly_1+=(gf_degree-len(poly_1))*[0]
    
    # if len(poly_2)!=gf_degree:
    #     poly_2+=(gf_degree-len(poly_2))*[0]


    # perform regular polynomial multiplication
    # cast to integer coefficients
    poly_prod = Poly.polymul(poly_1, poly_2).astype(int)

    # apply "mod 2" operation to each coefficient
    poly_prod = modCoeffs(poly_prod, 2)

    # perform division between polynomials
    # use the corresponding irreducible polynomial 
    # as the divisor polynomial
    # this returns a quotient polynomial and 
    # a remainder polynomial
    temp, poly_mod = Poly.polydiv(poly_prod, modulus)

    # apply "mod 2" operation to each coefficient
    poly_mod = modCoeffs(poly_mod, 2).astype(int)


    if len(poly_mod)!=gf_degree:
        i = len(poly_mod)
        while i!=gf_degree:
            poly_mod = np.append(poly_mod, 0)
            i+=1

    return poly_mod

def xor(x,y):
    if type(x) == str:
        return "".join([chr(ord(a) ^ ord(b)) for a,b in zip(x,y)])
    elif type(x) == bytes:
        return "".join([chr(a ^ b) for a,b in zip(x,y)]).encode('iso-8859-1')

def pad(ptxt):
    # we add padding so each block is 64bits.
    if len(ptxt)% 8 !=0:
        size = 8 - (len(ptxt)%8)
        ptxt += size*b'\x00'
    return ptxt
    
def bytes_to_poly(block):
    # Converts block into galois field polynomial array
    # Each index is one bit of the 64 bit number
    assert (len(block) ==  8)
    binary_string = ''.join(format(byte, '08b') for byte in block)
    poly = [int(bit) for bit in binary_string]
    return poly

def poly_to_bytes(poly):
     binary_string = ''.join(map(str, poly))
     return bytes(int(binary_string[i:i+8], 2) for i in range(0, len(binary_string), 8))
    

In [90]:
# 3 points
def generate_s(K2: int) -> int:
    """
    Compute s = Simon64/128(K2,0x00) and return it.
    Use the correct mode of operation for a single block.
    """
    s = 0
    # YOUR CODE HERE
    cipher=SimonCipher(K2,key_size=128,block_size=64,mode='ECB') # simon cipher for single block
    zero_block=b'\x00' * 16 # zero block
    encrypted_block=cipher.encrypt(bytes_to_long(zero_block)) # encrypt block
    s=encrypted_block 

    return s

# 2 points
def length_encoding(lengthA: int, lengthM: int) -> bytes:
    """
    The 32 MSBs (most significant bits) of length_block 
    encode the number of blocks of A, and the 32 LSBs (least significant bits)
    of length_block encode the number of blocks of M.
    Encode each as big endian.
    """
    length_block = b""
    # YOUR CODE HERE
    length_block=(lengthA << 32 | lengthM).to_bytes(8, byteorder='big') #replace A with block of M
    return length_block

# 10 points
def bhash64(s: int, A: bytes, M: bytes) -> bytes:
    """
    Implement BHASH64.
    Use the poly helper functions provided above. return digest
    in bytes format.
    Hints: 
    convert s to poly using bytes_to_poly()
    The initial value (X0) should be all 0s.
    First hash a padded A, then hash a padded M. Use bytes_to_poly() on each block
    Finally, hash a length encoding block using length_encoding()
    Finally, convert the digest back to bytes using the poly_to_bytes function
    """
    digest = b""
    # YOUR CODE HERE
    gf_degree=64 
    modulus=[1] + [0] * 63 + [1]  #p(x)=x^64+1
    s_poly=bytes_to_poly(s.to_bytes(8, byteorder='big')) # s is converted polynomial
    hash_poly=[0] * gf_degree # initialise hash to zero
    A_pad=pad(A) # pad A
    for i in range(0, len(A_pad), 8):
        block=A_pad[i:i+8] # get block of A padded
        block_poly=bytes_to_poly(block) #convert block to polynomial
        hash_poly=poly_xor(hash_poly, block_poly) #xor hash polynomial with block polynomial
        hash_poly=poly_mul(hash_poly, s_poly, gf_degree, modulus) #muliply hash polynomial with s and take modulas 
    M_pad=pad(M) # pad M
    for i in range(0, len(M_pad), 8):
        block=M_pad[i:i+8]
        block_poly=bytes_to_poly(block) #convert block to polynomial
        hash_poly=poly_xor(hash_poly, block_poly) #xor hash polynomial with block polynomial
        hash_poly=poly_mul(hash_poly, s_poly, gf_degree, modulus) #muliply hash polynomial with s and take modulas 
    lengthA=len(A) # get length of A
    lengthM=len(M) # get length of M
    length_block=length_encoding(lengthA, lengthM) #encode lengths of A and M
    length_block_poly=bytes_to_poly(length_block) # convert to polynomial
    hash_poly=poly_xor(hash_poly, length_block_poly) # xor hash polynomial with length encoding polynomial
    hash_poly=poly_mul(hash_poly, s_poly, gf_degree, modulus) # multiply with s and take modulas
    digest=poly_to_bytes(hash_poly) #convert to bytes
    return digest

In [91]:
K2 = 1023452506668115623771221871214768225
s = generate_s(K2)
assert s == 15063836755188045411

In [92]:
assert length_encoding(472, 672) == b'\x00\x00\x01\xd8\x00\x00\x02\xa0'

In [93]:
M = b"I'm a good student that createsptxtsize of more than 16"
A = b"extra data"
K2 = 1023452506668115623771221871214768225
digest = bhash64(generate_s(K2), A, M)
assert digest == b'\x9b\xfa\xfc\xb7\x8bl6v'

## (c) [10 points] Implement the C-W MAC(K2,N,A,M) that generates tags T = Simon64/128-ECB(K2, N xor BHASH64(s,A,M)), where K2 is a key, N is a nonce, A is associated data blocks and M is plaintext blocks.

## [2 points] Why did we choose to define T = Simon64/128(K2, N xor BHASH64(s,A,M) ) instead of T = BHASH64(s,A,M) xor Simon64/128(K2, N || 0x0000…00)? Why the former definition is better?

Xoring the nonce with the hash makes it harder to forge or predict the tag hence why we chose to define T = Simon64/128-ECB(K2, N xor BHASH64(s,A,M))

In [94]:
# 8 points
def CWMAC(K2: int, N: int, A: bytes, M: bytes) -> bytes:
    """
    Implement the C-W MAC. Call the bhash64(s, A, M) function and use 
    the Simon cipher to generate the tag based on the question
    description. Return tag in bytes.
    Hints: Use bytes_to_long and long_to_bytes.
    """
    tag = b""
    # YOUR CODE HERE
    s=generate_s(K2) # generate s
    bhash_result=bhash64(s, A, M) # compute bhash
    nonce_bytes=long_to_bytes(N, 8)  #convert nonce to bytes
    xor=bytes_to_long(bhash_result) ^ bytes_to_long(nonce_bytes) # xor bhash with nonce
    cipher=SimonCipher(K2, key_size=128, block_size=64, mode='ECB') # initialise Simon cipher
    tag=long_to_bytes(cipher.encrypt(xor), 8) #compute tag
    
    return tag

In [95]:
M = b"I'm a good student that createsptxtsize of more than 16"
A = b"extra data"
N = 4347369957267604693
K2 = 92421660963770782211016436480228233283
tag = CWMAC(K2, N, A, M)
assert tag == b'\xfeV\xf11\xd8\xee\xf3+'

## (d) [10 points] Implement the GCM-SIV mode variant for AEAD that combines the cipher from part (a) with the tag from part (c) ensuring correct SIV combination of the two primitives (i.e., as described in construction 3.1 of the paper cited above). Include decryption & tag verification support. Finally, state any necessary assumptions you made regarding the counters.

In [96]:
# 5 points
def gcm_encrypt(K1: int, K2: int, N: int, A: bytes, M: bytes) -> (bytes,bytes):
    """
    Implement the encryption function of GCM-SIV.
    Use K1 to genrate ctxt C using simon_encrypt.
    Use K2 to generate tag T using CWMAC function.
    return pair of ctxt, tag (C,T) in bytes format.
    """
    T = b""
    C = b""
    # YOUR CODE HERE
    C=simon_encrypt(K1, N, M)  #ptxt is encrypted with Simon 64/128 CTR mode
    T=CWMAC(K2, N, A, M) #generate tag
    return (C, T)
# 5 points
def gcm_decrypt(K1: int, K2: int, N: int, A: bytes, C: bytes, T: bytes) -> (bytes,bool):
    """
    Implement the decryption function of GCM-SIV. 
    Use K1 to generate tag T_prime using CWMAC function.
    Use K2 to genrate ptxt M using simon_decrypt.
    If T_prime equals to T, reutrn (M,True) if not return (b"",False).
    """
    verified = True
    M = b""
    
    # YOUR CODE HERE
    M = simon_decrypt(K1, N, C) # decrypt message
    T_prime = CWMAC(K2, N, A, M) # compute T_prime
    if T_prime != T:
        return (b"", False)
    
    return (M,verified)

In [97]:
M = b"I'm a good student that createsptxtsize of more than 16"
A = b"extra data"
N = 471552568800831418
K1 = 272968999337452403437395976493262077421
K2 = 98065785485843554162410036472004181817

C , T = gcm_encrypt(K1, K2, N, A, M)
assert C == b'\xf8i\x9e}@\xe0\\A\xcd\xc5\x81\xf4K\x05L\xc2\x07h&\x9f\xeab\x9dqz\xa9\xd4q\xefA\x12\xafg\xd3\x91\xec\x05>\xbc\xf2\x11\x90\xd9W \xd7\xe4\xca|\x01]Zh\xe1\xf2\x8b'
assert T == b'Z&\xe8 ,9i\x1e'

In [98]:
M = b"I'm a good student that createsptxtsize of more than 16"
A = b"extra data"
N = 471552568800831418
K1 = 272968999337452403437395976493262077421
K2 = 98065785485843554162410036472004181817
C , T = gcm_encrypt(K1, K2, N, A, M)
M_decrypt , verified = gcm_decrypt(K1, K2, N, A, C, T)
assert M_decrypt == M
assert verified == True 

AssertionError: 